# Text 4: Word2Vec
**Internet Analytics - Lab 4**

---

**Group:** *Your group letter.*

**Names:**

* *TAY Ying Xu Dempster*
* *LIU Linqi*
* *YAN Yuhang*
* *SONG Yifei*
---

#### Instructions

*This is a template for part 4 of the lab. Clearly write your answers, comments and interpretations in Markodown cells. Don't forget that you can add $\LaTeX$ equations in these cells. Feel free to add or remove any cell.*

*Please properly comment your code. Code readability will be considered for grading. To avoid long cells of codes in the notebook, you can also embed long python functions and classes in a separate module. Don’t forget to hand in your module if that is the case. In multiple exercises, you are required to come up with your own method to solve various problems. Be creative and clearly motivate and explain your methods. Creativity and clarity will be considered for grading.*

In [25]:
import pickle
import re
import numpy as np
from scipy.sparse import csr_matrix
from collections import defaultdict
import json
from utils import *
import gensim
from sklearn.cluster import KMeans


courses = load_json('data/courses.txt')
stopwords = load_pkl('data/stopwords.pkl')

## Redo pre-processing

In [26]:
print(courses[0])

{'courseId': 'MSE-440', 'name': 'Composites technology', 'description': "The latest developments in processing and the novel generations of organic composites are discussed. Nanocomposites, adaptive composites and biocomposites are presented. Product development, cost analysis and study of new markets are practiced in team work. Content Basics of composite materialsConstituentsProcessing of compositesDesign of composite structures\xa0Current developmentNanocomposites Textile compositesBiocompositesAdaptive composites\xa0ApplicationsDriving forces and marketsCost analysisAerospaceAutomotiveSport Keywords Composites - Applications - Nanocomposites - Biocomposites - Adaptive composites - Design - Cost Learning Prerequisites Required courses Notion of polymers Recommended courses Polymer Composites Learning Outcomes By the end of the course, the student must be able to: Propose suitable design, production and performance criteria for the production of a composite partApply the basic equati

In [27]:
import string
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [28]:
# Split connected words (eg. materialsConstituentsProcessing but not for acronyms like IT)

for course in courses:
	words = []
	current_word = ''

	for word in course['description'].split():
		for char in word:
			# If char is uppercase, it is the start of a new word
			if char.isupper():
				# Don't add word yet if it is all uppercase (acronym)
				if current_word.isupper() and len(current_word) > 0:
					current_word += char
					continue
				# Add current word
				words.append(current_word)
				current_word = char
			else:
				current_word += char

		words.append(current_word)
		current_word = ''

	course['description'] = ' '.join(words)

In [29]:
# Remove punctuation

for course in courses:
	course['description'] = ''.join([char for char in course['description'] if char not in string.punctuation])

In [30]:
# Remove stopwords and capitalized versions

stopwords_capitalized = [s.capitalize() for s in stopwords]

for course in courses:
	course['description'] = ' '.join([w for w in course['description'].split() if w not in stopwords and w not in stopwords_capitalized])

In [31]:
# Check word frequency

freq_dict = {}

for course in courses:
  for word in course['description'].split():
    if word in freq_dict:
      freq_dict[word] += 1
    else:
      freq_dict[word] = 1

In [32]:
frequent_sorted = sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)
infrequent_sorted = sorted(freq_dict.items(), key=lambda x: x[1])
print(frequent_sorted[:10])
print(infrequent_sorted[:10])

[('methods', 1592), ('Learning', 1239), ('student', 1177), ('Content', 835), ('courses', 757), ('systems', 697), ('end', 661), ('students', 655), ('design', 603), ('Outcomes', 600)]
[('biocomposites', 1), ('Constituents', 1), ('Textile', 1), ('Automotive', 1), ('Sport', 1), ('photograph', 1), ('blot', 1), ('extracted', 1), ('reproducibility', 1), ('pipelines', 1)]


In [33]:
# Choose infrequent words that only appear once

infrequent_sorted = [w for w in infrequent_sorted if w[1] == 1]
len(infrequent_sorted)

# Make lists of frequent and infrequent words

frequent_list = [w[0] for w in frequent_sorted[:10]]
infrequent_list = [w[0] for w in infrequent_sorted]

In [34]:
# Remove frequent and infrequent words

for course in courses:
	course['description'] = ' '.join([w for w in course['description'].split() if w not in frequent_list and w not in infrequent_list])

print(courses[0]['description'])

latest developments processing generations organic composites discussed Nanocomposites adaptive composites presented Product development cost analysis study markets practiced team work Basics composite materials Processing composites Design composite structures Current development Nanocomposites composites Biocomposites Adaptive composites Applications Driving forces markets Cost analysis Aerospace Keywords Composites Applications Nanocomposites Biocomposites Adaptive composites Design Cost Prerequisites Required Notion polymers Recommended Polymer Composites Propose suitable production performance criteria production composite part Apply basic equations process mechanical properties modelling composite materials Discuss main types composite applications Transversal skills work methodology task general domain specific IT resources tools Communicate effectively professionals disciplines Evaluate performance team receive respond appropriately feedback Teaching cathedra invited speakers G

**We don't tranform all the words to lowercase, and we don't perform stemming
or lemmatization compared to pre-processing in Exercise 4.1.**

## Exercise 4.12 : Clustering word vectors

**Here we try to find the most frequent word that is in `courses['description']` and the word list of Word2Vec model. The most frequent word will work as a default word for the words in `courses['description']` and not present in the model.**

In [35]:
from collections import Counter
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity

def most_frequent_word_in_description(words, model_path):
    # Filter out words not present in the Word2Vec model
    valid_words = [word for word in words if word in model]

    word_counts = Counter(valid_words)

    # Find the most frequent word
    if word_counts:
        most_frequent_word = word_counts.most_common(1)[0][0]
        return most_frequent_word
    else:
        return None

model = gensim.models.KeyedVectors.load_word2vec_format('/ix/model.txt')
all_words = []
for course in courses:
    all_words.extend(course['description'].split())
most_frequent_word = most_frequent_word_in_description(all_words, model)
print(most_frequent_word)

analysis


**If it cannot find a word in the model, it will return the vector of the default word.**

In [36]:
def get_vector(word):
    try:
        return model[word]
    except KeyError:
        return model[most_frequent_word]

In [37]:
# word_vectors = {word: get_vector(word) for word in all_words}
word_vectors = dict(map(lambda word: (word, get_vector(word)), all_words))

# Normalize the word vectors
vectors = np.array(list(word_vectors.values()))
normalized_vectors = normalize(vectors)

**Perform KMeans algorithm**

In [38]:
num_clusters = 20
kmeans = KMeans(n_clusters=num_clusters, random_state=0, init='k-means++')
kmeans = kmeans.fit(normalized_vectors)


**Print the top-10 words in each cluster**

In [39]:
def get_top_words_in_clusters(kmeans, word_vectors, top_n=10):
    cluster_centroids = kmeans.cluster_centers_
    top_words = {}
    
    words = list(word_vectors.keys())
    vectors = np.array(list(word_vectors.values()))
    labels = kmeans.labels_

    for cluster_idx in range(len(cluster_centroids)):
        centroid = cluster_centroids[cluster_idx]
        
        # Get the indices of word vectors that belong to the current cluster
        cluster_word_indices = [i for i, label in enumerate(labels) if label == cluster_idx]
        cluster_words = [words[i] for i in cluster_word_indices]
        cluster_vectors = [vectors[i] for i in cluster_word_indices]

        if cluster_vectors:
            similarities = model.cosine_similarities(centroid, cluster_vectors)
            word_similarity_pairs = zip(cluster_words, similarities)
            
            sorted_words = sorted(word_similarity_pairs, key=lambda x: x[1], reverse=True)
            top_words[cluster_idx] = [word for word, _ in sorted_words[:top_n]]
        else:
            top_words[cluster_idx] = []

    return top_words

top_words = get_top_words_in_clusters(kmeans, word_vectors)
for cluster_idx, words in top_words.items():
    print(f"Cluster {cluster_idx}: {', '.join(words)}")

Cluster 0: structures, adjacent, areas, connecting, encompassing, stretching, connected, surrounding, area, complex
Cluster 1: neuronal, metabolic, vitro, biochemical, hematopoietic, metabolism, angiogenesis, inhibition, bacterial, tissues
Cluster 2: Nanocomposites, analysis, Biocomposites, Prerequisites, Propose, IT, timelapse, FIJI, Illustrate, Integrate
Cluster 3: Implementation, Analysis, Evaluation, Methodology, Policies, Assessment, Context, Policy, Strategy, Integration
Cluster 4: thin, vertical, horizontal, mesh, surfaces, stacked, cylindrical, metallic, stretchable, thick
Cluster 5: Baker, Smith, Robinson, Taylor, Freeman, Patterson, Meyer, Cummings, Evans, Rosenthal
Cluster 6: seminars, lectures, presentations, teaching, seminar, lecturers, tutorials, symposia, instructors, professors
Cluster 7: software, interfaces, implementations, Simulink, functionality, APIs, interface, opensource, debugging, scalable
Cluster 8: quadratic, ODEs, equations, discretized, finite, summabilit

**number of clusters:**

We consider 20 to be a good choice of numnber of cluster, because EPFL has 18 sections and we may choose k value close to 18. With k=20, we can preserve the diversity of the clusters, while not having different clusters for similar words.

**We observe the following types of clusters: structure analyses, Biology, Material Science, policies and strategy, education, description and charateristic, finance, names and identities, subjects, mathematics, numbers, verbs, environment sciences.**

**Here we give labels to the representative cluster of its type**

1. Structure Analyses: Cluster 0

2. Biology: Cluster 1

3. Material Science and Chemistry: Cluster 2

4. Names and Identity: Cluster 5

5. Education: Cluster 6

6. Software development: Cluster 7

7. Mathematics: Cluster 8

8. Finance and Management: Cluster 15

9. Physics: Cluster 17

10. Environment Science: Cluster 18

**Compared to LSI and LDA:**

**With LSI**:

W2V's results are more consistent within the same cluster, while LSI's seems not clustering well. LSI's results contain some topics unseen in W2V's results, such as "Philosophy", "Wireless", and "Security". Still, the two methods get a few similar topics, such as "Market" and "Finance and Management", "Chemistry" and "Material Science and Chemistry".

**With LDA**:

Both of them get topics like "Biology", "Chemistry", "Management"; some topics from the results are not exactly the same but are quite related, such as "Environment Science" and "Urban Development", "Mathematics" and "Optics and Computation". Besides, W2V's results contain topic like "Names and Identity", which is not seen in both LSI and LDA.

## Exercise 4.13 : Document similarity search

**Implement an aggregation of the `word_vectors` using the weighted average, whose weights are the TF-IDF scores of the corresponding words.
Use the vector aggregation to perform a similarity search with cosine similarity**

**Reuse the model in 4.2**

In [40]:
# Keep track of the mapping between terms and their indices, and documents and their indices
terms_indices = list(set(all_words))

document_indices = [x['courseId'] for x in courses]

In [41]:
# Construct matrix X

M = len(terms_indices)
N = len(document_indices)

X = np.zeros((M, N))
X.shape

(9324, 854)

In [42]:
# Populate matrix X

for i, word in enumerate(terms_indices):
  for j, course in enumerate(courses):
    description_list = course['description'].split()
    freq = description_list.count(word)
    X[i][j] = freq / len(description_list)

In [43]:
# Construct IDF

idf = np.zeros(len(terms_indices))

for row in range(len(terms_indices)):
	freq = 0
	for val in X[row]:
		if val > 0:
			freq += 1
	idf[row] = np.log(len(courses) / freq)

idf = idf.reshape(len(idf), 1)
idf

array([[4.95817172],
       [6.05678401],
       [3.91671785],
       ...,
       [6.05678401],
       [6.05678401],
       [6.05678401]])

In [44]:
# Construct TF-IDF matrix

tf_idf = X * idf

**Compute the weighted average embedding for each document(course).**

In [55]:
document_represent = np.zeros((N, 300))

for j in range(N):
    description_list = courses[j]['description'].split()
    weighted_vectors = []
    for word in description_list:
        word_idx = terms_indices.index(word)
        word_vector = word_vectors[word]
        # Weighted word vector
        word_vector = tf_idf[word_idx, j]*word_vector
        weighted_vectors.append(word_vector)
    weighted_average = sum(weighted_vectors)/len(weighted_vectors)
    document_represent[j, :] = weighted_average
document_represent

array([[ 1.48352061e-03,  4.32334171e-04,  2.02237279e-03, ...,
        -2.25496688e-03, -6.04416186e-04,  1.57838411e-04],
       [ 1.12673652e-03, -2.88368465e-04,  1.68587675e-03, ...,
         6.62948005e-04,  5.71249984e-04,  6.30495779e-04],
       [-3.49563314e-04, -3.48699032e-05,  8.84284440e-04, ...,
         3.41499341e-04,  5.61502064e-04,  6.89147055e-05],
       ...,
       [ 1.88111444e-04,  4.61974239e-04,  9.05692170e-04, ...,
         6.93232461e-04,  2.11196657e-05,  2.76320381e-04],
       [ 7.28788972e-03, -1.33930138e-04,  8.71425867e-03, ...,
         2.79346434e-03,  1.26116828e-03,  1.17385818e-03],
       [ 2.88377260e-03,  1.63334000e-04,  2.61979504e-03, ...,
        -3.80084603e-05,  8.05529184e-04,  1.05522841e-03]])

**Compute vectors representation for two queries**

In [72]:
# Markov chains
vectors = []
for word in ['Markov', 'chains']:
    word_vector = get_vector(word)
    # Perform average without weighting
    vectors.append(word_vector)
markov_chain_represent = sum(vectors)/len(vectors)

# Facebook
facebook_represent = get_vector('Facebook')

In [47]:
# Function to compare two documents

def cosine_similarity(doc1, doc2):
	norm_doc1 = np.linalg.norm(doc1)
	norm_doc2 = np.linalg.norm(doc2)
	return np.dot(doc1, doc2) / (norm_doc1 * norm_doc2)

In [62]:
# Search for "Markov chains"

markov_similarity = []
for idx in range(len(document_indices)):
  markov_similarity.append(cosine_similarity(document_represent[idx], markov_chain_represent))

# Dictionary {course name: similarity value}
markov_sim_vals = {}
for idx, sim in enumerate(markov_similarity):
  markov_sim_vals[courses[idx]['name']] = sim

# Sort by similarity value
markov_sim_vals_sorted = sorted(markov_sim_vals.items(), key=lambda x:x[1], reverse = True)


In [63]:
# Search for "Facebook"

facebook_similarity = []
for idx in range(len(document_indices)):
    facebook_similarity.append(cosine_similarity(document_represent[idx], facebook_represent))

# Dictionary {course name: similarity value}
facebook_sim_vals = {}
for idx, sim in enumerate(facebook_similarity):
    facebook_sim_vals[courses[idx]['name']] = sim

# Sort by similarity value
facebook_sim_vals_sorted = sorted(facebook_sim_vals.items(), key=lambda x:x[1], reverse = True)

**Display the top five courses together with their similarity score for each query**

In [65]:
# markov chain
print("Markov chains - top five courses with similarity score")
for i in range(5):
    print(markov_sim_vals_sorted[i])

# facebook
print("\nFacebook - top five courses with similarity score")
for i in range(5):
    print(facebook_sim_vals_sorted[i])

Markov chains - top five courses with similarity score
('Applied probability & stochastic processes', 0.7490182486824046)
('Applied stochastic processes', 0.7008952611238507)
('Markov chains and algorithmic applications', 0.6525220430300869)
('Statistical Sequence Processing', 0.6281523846854378)
('Mathematical models in supply chain management', 0.5444954972033781)

Facebook - top five courses with similarity score
('Computational Social Media', 0.5967716881078419)
('Human computer interaction', 0.4811589492949727)
('Social media', 0.4639274464913632)
('Internet analytics', 0.460725455455456)
('Image communication', 0.4440662471067638)


**Compare with what you obtain for the vector space model and LSI:**

The top five courses obtained for this vector space model are similar to what we obtained by LSI. The result for searching 'Markov chains' and 'Facebook' seems more reasonable and related compared to the result from LSI. For 'Facebook', LSI only identifies 1 related course 'Computational Social Media', and assigns 0 similarity score for all other courses. There may be two reasons for this: 

1. we can preserve the uppercases using the word2vec model, and thus preserve the meaning of some case sensitive terms; 
2. we use the weighted average as a document representation which give us a more accurate representation;
3. Word2Vec can handle synonyms better due to its contextual embeddings. For example, it understands that 'Facebook' and 'social media platform' are related. LSI, on the other hand, might treat these terms as separate, so it cannot find anything related in most courses.

## Exercise 4.14: Document similarity search with outside terms

**Search for "MySpace Orkut"**

In [71]:
vectors = []
for word in ['MySpace', 'Orkut']:
    word_vector = get_vector(word)
    # Perform average without weighting
    vectors.append(word_vector)
myspace_orkut_represent = sum(vectors)/len(vectors)

In [79]:
# Search for "MySpace Orkut"

myspace_orkut_similarity = []
for idx in range(len(document_indices)):
  myspace_orkut_similarity.append(cosine_similarity(document_represent[idx], myspace_orkut_represent))

# Dictionary {course name: similarity value}
myspace_orkut_sim_vals = {}
for idx, sim in enumerate(myspace_orkut_similarity):
  myspace_orkut_sim_vals[courses[idx]['name']] = sim

# Sort by similarity value
myspace_sim_vals_sorted = sorted(myspace_orkut_sim_vals.items(), key=lambda x:x[1], reverse = True)

**display the top five results**

In [80]:
print("MySpace Orkut - top five courses with similarity score")
for i in range(5):
    print(myspace_sim_vals_sorted[i])

MySpace Orkut - top five courses with similarity score
('Computational Social Media', 0.5576601987460832)
('Human computer interaction', 0.4990797492983756)
('Social media', 0.48222115789269315)
('Internet analytics', 0.47391868515121005)
('Image communication', 0.46852260201614826)


**Compare the results with what you obtained for "Facebook" in the previous exercise:**

The results are nearly identical, except some differences in the similarity score. The results make sense because "MySpace" and "Orkut" were both early social networking platforms that predated Facebook. They are all the similar type of applications, and it's very likely that they will be mentioned in the same courses that talks about social media and Internet, e.g. Computational Social Media.

**Search for "coronavirus"**

In [85]:
coronavirus_represent = get_vector('coronavirus')

In [88]:
# Search for "coronavirus"

coronavirus_similarity = []
for idx in range(len(document_indices)):
  coronavirus_similarity.append(cosine_similarity(document_represent[idx], coronavirus_represent))

# Dictionary {course name: similarity value}
coronavirus_sim_vals = {}
for idx, sim in enumerate(coronavirus_similarity):
  coronavirus_sim_vals[courses[idx]['name']] = sim

# Sort by similarity value
coronavirus_sim_vals_sorted = sorted(coronavirus_sim_vals.items(), key=lambda x:x[1], reverse = True)

**Display top 5 results**

In [89]:
print("coronavirus - top five courses with similarity score")
for i in range(5):
    print(coronavirus_sim_vals_sorted[i])

coronavirus - top five courses with similarity score
('Infection biology', 0.5655043101774891)
('Biotechnology lab (for CGC)', 0.5200209800804626)
('Practical - Blokesch Lab', 0.5182632365497325)
('Practical - Lemaitre Lab', 0.5111887713254325)
('Neurosciences I : molecular neuroscience and neurodegeneration', 0.5021366636642971)


**Do the results seem reasonable?**

The results seem very reasonable. The searching outputs contain Biology related courses. "Infection biology" has highest similarity score, and it's very related to infectious disease like coronavirus.